In [ ]:
import numpy as np
import geopandas as gp
import matplotlib.pyplot as plt
from sklearn.cluster import OPTICS, cluster_optics_dbscan
import time
import networkx as nx
import pandas as pd
import math
from math import radians, sin, cos, asin, sqrt
from scipy.spatial import Delaunay
from shapely.geometry import LineString
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Enter the right datafile
buildings = 'SLE_KenemaRadius_OSMBuildings.geojson'

print('Loading data...')
# Prepare building data - read as geodataframe
gdf = gp.read_file(buildings)
# Get the number of rows (points) in the GeoDataFrame
no_points = gdf.shape[0]
gdf.head()
print(no_points)


In [ ]:
# Get centroid points of building polygons
gdf['centroid'] = gdf['geometry'].centroid
# Break out lat and long into separate columns of GeoDataFrame
gdf['lon'] = gdf.centroid.x
gdf['lat'] = gdf.centroid.y
# Get lat and long columns from the GeoDataFrame and convert into a numpy array
coords = gdf.drop(['name', 'type', 'code', 'fclass', 'osm_id', 'geometry', 'centroid'], axis=1).to_numpy()

In [ ]:
# Plot home locations
plt.figure(figsize=(7,7))
plt.title('Home locations')
plt.scatter(x=coords[:, 0], y=coords[:, 1],color='b', s=3, alpha=0.8)
plt.xlabel('Longitude ($^\circ$)')
plt.ylabel('Latitude ($^\circ$)')
axes = plt.gca()
axes.set_xlim([min(coords[:, 0]) - 0.001, max(coords[:, 0]) + 0.001])
axes.set_ylim([min(coords[:, 1]) - 0.001, max(coords[:, 1]) + 0.001])


In [ ]:
# Cluster the data

clust = OPTICS(min_samples= 10, xi=.7, min_cluster_size= 10)

# Run the fit
clust.fit(coords)
labelsOp = clust.labels_[clust.ordering_]

# See how many houses in each cluster. cluster -1 = outlier
(unique, counts) = np.unique(labelsOp, return_counts=True)
frequencies = np.asarray((unique, counts)).T
print(frequencies)

#plot results
plt.figure(figsize=(7, 7))

labels = labelsOp
for klass in zip(range(0, 297)):
    plt.scatter(x=coords[labels == klass, 0], y=coords[labels == klass, 1],color=np.random.rand(3), s=5, alpha=0.8)


In [ ]:
# Use DBSCAN method

labelsF = cluster_optics_dbscan(reachability=clust.reachability_,
                                   core_distances=clust.core_distances_,
                                   ordering=clust.ordering_, eps=0.002)

# See how many houses in each cluster. -1 = outlier
(unique, counts) = np.unique(labelsF, return_counts=True)
frequencies = np.asarray((unique, counts)).T
print(frequencies)

plt.figure(figsize=(7, 7))

labels = labelsF
for klass in zip(range(0, 297)):
    plt.scatter(x=coords[labels == klass, 0], y=coords[labels == klass, 1],color=np.random.rand(3), s=5, alpha=0.8)

#plot outliers
# plt.scatter(x=coords[clust.labels_ == -1, 0], y=coords[clust.labels_ == -1, 1],color='k', s=1, alpha=0.8)

In [ ]:
# Haversine formula for kilometer distance between two lat/long points
def haversine_dist_from_coords(lat1, lon1, lat2, lon2):
    # The math module contains a function named radians which converts from degrees to radians.
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * asin(sqrt(a))
    # Radius of earth in kilometers. Use 3956 for miles
    r = 6371
    # calculate and return the result
    return c * r

In [ ]:
#For each cluster, perform delaunay triangulation and save the result as a shp file to be able to visualise
#final cabling layout in QGIS

total_distance_under100 = 0
total_distance_100_300 = 0
total_distance_300_500 = 0
total_distance_1000 = 0
clusters_under_100 = 0
clusters_100_300 = 0
clusters_300_500 = 0
clusters_1000 = 0

for klass in range(0,298):
    newcoords = coords[labels ==klass]
    
    no_points = len(newcoords)
    print(no_points)
    
    print('Calculating Delaunay triangulation and distance between Delaunay neighbours...')
    # Get Delauney triangulation of coordinates
    tri = Delaunay(newcoords)
    indices = tri.vertex_neighbor_vertices[0]
    indptr = tri.vertex_neighbor_vertices[1]
    
    # Instantiate dictionary to hold neighbors of each point & data-frame to hold distances between neighbours
    neighbors = {}
    locations = {}
    distances = pd.DataFrame(columns=["source", "dest", "distance"])
    
    # Get dictionary of neighbors of all points and a dictionary of locations of all points
    for k in range(0, no_points):
        neighbors[k] = indptr[indices[k]:indices[k+1]]
        locations[k] = newcoords[k][0], newcoords[k][1]
    
    # Get distances between all Delaunay neighbors
    for key, values in neighbors.items():
        for value in values:
            coord_1 = newcoords[key]
            coord_2 = newcoords[value]
            dist = haversine_dist_from_coords(coord_1[1], coord_1[0], coord_2[1], coord_2[0])
            distances = distances.append({"source": key, "dest": value, "distance": dist}, ignore_index=True)
            
    # Plot Delaunay triangulation
    plt.figure(figsize=(7, 7))
    plt.title('Delaunay Triangulation of Homes')
    plt.triplot(newcoords[:, 0], newcoords[:, 1], tri.simplices)
    plt.xlabel('Longitude ($^\circ$)')
    plt.ylabel('Latitude ($^\circ$)')
    plt.plot(newcoords[:, 0], newcoords[:, 1], 'o')
    axes = plt.gca()
    axes.set_xlim([min(newcoords[:, 0]) - 0.001, max(newcoords[:, 0]) + 0.001])
    axes.set_ylim([min(newcoords[:, 1]) - 0.001, max(newcoords[:, 1]) + 0.001])
    
    print('Creating a graph from this information (edge weight = distance)...')
    G = nx.Graph()
    for index, row in distances.iterrows():
        G.add_edge(row['source'], row['dest'], weight=row['distance'])
        
    print('Calculating the minimum spanning tree of the graph...')
    T = nx.minimum_spanning_tree(G)
        
    edges = T.edges(data=True)
    weights = [x[2]['weight'] for x in edges]
    total_dist = sum(weights)*1000
        
    print('Number of nodes (buildings) in the graph: ', T.number_of_nodes())
    print('Number of edges in the minimum spanning tree: ', T.number_of_edges())
    print('Total distance of minimum spanning tree (in m): ', total_dist)
    
    
    #The section below simply allows to have a rough estimate of the number of clusters and cabling distances
    #depending on their size:
    if 0 < no_points <= 100:
        clusters_under_100 = clusters_under_100 + 1
        total_distance_under100 = total_distance_under100 + total_dist
    elif 100 < no_points <= 300:
        clusters_100_300 = clusters_100_300 + 1
        total_distance_100_300 = total_distance_100_300 + total_dist
    elif 300 < no_points <= 500:
        clusters_300_500 = clusters_300_500 + 1
        total_distance_300_500 = total_distance_300_500 + total_dist
    elif 500 < no_points:
        clusters_1000 = clusters_1000 + 1
        total_distance_1000 = total_distance_1000 + total_dist
    
    #Create a geopandas dataframe and save as .shp

    # create an array of LineString from T
    lines = [LineString([(newcoords[int(edge[0]),0],newcoords[int(edge[0]),1]),(newcoords[int(edge[1]),0],newcoords[int(edge[1]),1])]) for edge in edges]

    d = {'geometry':lines}
    mstDF = gp.GeoDataFrame(d, crs="EPSG:4326")
    mstDF.to_file("MST"+str(klass)+".shp")

        
    print('Plotting results:')
        
    # Plot Minimum Spanning Tree made from Delaunay Triangulation
    plt.figure(figsize=(12, 12))
    nx.draw_networkx(T, pos=locations, with_labels=False, node_size=15)
    plt.title('Minimum Spanning Tree of Delaunay Graph \n (Edge Weight = Haversine Distance)')
    plt.xlabel('Longitude ($^\circ$)')
    plt.ylabel('Latitude ($^\circ$)')
    axes = plt.gca()
    axes.set_xlim([min(newcoords[:, 0]) - 0.001, max(newcoords[:, 0]) + 0.001])
    axes.set_ylim([min(newcoords[:, 1]) - 0.001, max(newcoords[:, 1]) + 0.001])
        
    # Plot relative frequency of edge distances in minimum spanning tree
    plt.figure(figsize=(10, 4))
    plt.hist(weights, bins=200)
    plt.yscale("log")
    plt.ylabel('Number of edges of this distance')
    plt.xlabel('Distance (km)')

    plt.show()

print (f'under 100:{clusters_under_100} clusters and {total_distance_under100} m cabling')
print (f'100-300:{clusters_100_300} clusters and {total_distance_100_300} m cabling')
print (f'300-500:{clusters_300_500} clusters and {total_distance_300_500} m cabling')
print (f'1000:{clusters_1000} clusters and {total_distance_1000} m cabling')


In [ ]:
#The last step is to interconnect clusters together when desirable; below is the centroid-cluster method but 
#the k-nearest neighbor algorithm would be more accurate and should be implemented by a technical expert.
#Moreover, cabling along roads should be prioritized. This piece of code is simply to give a brief estimate
#of cabling distances.

In [ ]:
#Get centroid of each cluster
all_centroids = []
for klass in range(0,298):
    newcoords2 = coords[labels == klass]
    no_points = len(newcoords2)
    sum_x = 0
    sum_y = 0
    for k in range(0, no_points):
        sum_x = sum_x + newcoords2[k, 0]
        sum_y = sum_y + newcoords2[k, 1]
    centroid_x = sum_x / no_points
    centroid_y = sum_y / no_points
    all_centroids.append([centroid_x, centroid_y])

all_centroids = np.asarray(all_centroids)

print(all_centroids)

#Plot the cluster centroids
plt.figure(figsize=(10, 4))
plt.scatter(x=all_centroids[:,0], y=all_centroids[:,1],color='r', s=1, alpha=0.8)
plt.ylabel('Latitude')
plt.xlabel('Longitude')   
    

In [ ]:
# Get Delauney triangulation of cluster centroid coordinates
tri = Delaunay(all_centroids)
indices = tri.vertex_neighbor_vertices[0]
indptr = tri.vertex_neighbor_vertices[1]

# Instantiate dictionary to hold neighbors of each point & data-frame to hold distances between neighbours
neighbors = {}
locations = {}
distances = pd.DataFrame(columns=["source", "dest", "distance"])

# Get dictionary of neighbors of all points and a dictionary of locations of all points
for k in range(0, len(all_centroids)):
    neighbors[k] = indptr[indices[k]:indices[k+1]]
    locations[k] = all_centroids[k][0], all_centroids[k][1]

# Get distances between all Delaunay neighbors
for key, values in neighbors.items():
    for value in values:
        coord_1 = all_centroids[key]
        coord_2 = all_centroids[value]
        dist = haversine_dist_from_coords(coord_1[1], coord_1[0], coord_2[1], coord_2[0])
        distances = distances.append({"source": key, "dest": value, "distance": dist}, ignore_index=True)

# Plot Delaunay triangulation
plt.figure(figsize=(7, 7))
plt.title('Delaunay Triangulation of Cluster Centroids')
plt.triplot(all_centroids[:, 0], all_centroids[:, 1], tri.simplices)
plt.xlabel('Longitude ($^\circ$)')
plt.ylabel('Latitude ($^\circ$)')
plt.plot(all_centroids[:, 0], all_centroids[:, 1], 'o')
axes = plt.gca()
axes.set_xlim([min(all_centroids[:, 0]) - 0.001, max(all_centroids[:, 0]) + 0.001])
axes.set_ylim([min(all_centroids[:, 1]) - 0.001, max(all_centroids[:, 1]) + 0.001])

print('Creating a graph from this information (edge weight = distance)...')
G = nx.Graph()
for index, row in distances.iterrows():
    G.add_edge(row['source'], row['dest'], weight=row['distance'])

print('Calculating the minimum spanning tree of the graph...')
T = nx.minimum_spanning_tree(G)

edges = T.edges(data=True)
weights = [x[2]['weight'] for x in edges]
total_dist = sum(weights)

print('Number of nodes (buildings) in the graph: ', T.number_of_nodes())
print('Number of edges in the minimum spanning tree: ', T.number_of_edges())
print('Total distance of minimum spanning tree (in km): ', total_dist)

In [ ]:
print('Plotting results:')

# Plot Minimum Spanning Tree made from Delaunay Triangulation
plt.figure(figsize=(7, 7))
nx.draw_networkx(T, pos=locations, with_labels=False, node_size=15)
plt.title('Minimum Spanning Tree of Delaunay Graph \n (Edge Weight = Haversine Distance)')
plt.xlabel('Longitude ($^\circ$)')
plt.ylabel('Latitude ($^\circ$)')
axes = plt.gca()
axes.set_xlim([min(all_centroids[:, 0]) - 0.01, max(all_centroids[:, 0]) + 0.01])
axes.set_ylim([min(all_centroids[:, 1]) - 0.01, max(all_centroids[:, 1]) + 0.01])


plt.show()


In [ ]:
# Create a geopandas dataframe and save as .shp

# create an array of LineString from T
lines = [LineString([(all_centroids[int(edge[0]),0],all_centroids[int(edge[0]),1]),(all_centroids[int(edge[1]),0],all_centroids[int(edge[1]),1])]) for edge in edges]

d = {'geometry':lines}
mstDF = gp.GeoDataFrame(d, crs="EPSG:4326")
mstDF.to_file("intraclusterMST.shp")